In [2]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [3]:
device = torch.accelerator.current_accelerator() if torch.accelerator.is_available() else 'cpu'
print(f"Using {device} device")

Using cuda device


In [4]:
class NeuralNetwork(nn.Module):
  def __init__(self):
    super().__init__()
    self.flatten = nn.Flatten()
    self.linear_relu_stack = nn.Sequential(
        nn.Linear(28 * 28, 512),
        nn.ReLU(),
        nn.Linear(512, 512),
        nn.ReLU(),
        nn.Linear(512, 10)
    )

  def forward(self, x):
    x = self.flatten(x)
    logits = self.linear_relu_stack(x)
    return logits


In [5]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [6]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f'Predicted class: {y_pred}')

Predicted class: tensor([9], device='cuda:0')


In [7]:
input_image = torch.rand(3, 28, 28)
print(input_image.size())

torch.Size([3, 28, 28])


In [8]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


In [9]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


In [11]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.LeakyReLU()
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[0.3236, 0.0000, 0.0000, 0.0000, 0.0000, 0.1816, 0.0000, 0.0000, 0.0000,
         0.4423, 0.0000, 0.0000, 0.2048, 0.1326, 0.0000, 0.3808, 0.0000, 0.2846,
         0.1678, 0.0000],
        [0.0541, 0.0000, 0.0000, 0.0000, 0.0092, 0.1415, 0.1710, 0.0000, 0.0000,
         0.5664, 0.0000, 0.0000, 0.3845, 0.7855, 0.0000, 0.1167, 0.0000, 0.4433,
         0.1185, 0.0000],
        [0.3320, 0.0000, 0.0000, 0.0000, 0.0000, 0.1582, 0.1124, 0.0000, 0.2003,
         0.6049, 0.0000, 0.0000, 0.4433, 0.2366, 0.0000, 0.1960, 0.0384, 0.0983,
         0.1691, 0.0000]], grad_fn=<ReluBackward0>)


After ReLU: LeakyReLU(negative_slope=0.01)


In [12]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

In [13]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

In [18]:
print(f"Model structure: {model}\n\n")
for name, param in model.named_parameters():
  print(f'Layer: {name} \nSize: {param.size()} \nValues : {param[:2]} \n\n')


Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight 
Size: torch.Size([512, 784]) 
Values : tensor([[ 0.0134, -0.0026,  0.0145,  ...,  0.0216, -0.0306, -0.0285],
        [-0.0256,  0.0112,  0.0087,  ...,  0.0032,  0.0180, -0.0103]],
       device='cuda:0', grad_fn=<SliceBackward0>) 


Layer: linear_relu_stack.0.bias 
Size: torch.Size([512]) 
Values : tensor([ 0.0256, -0.0308], device='cuda:0', grad_fn=<SliceBackward0>) 


Layer: linear_relu_stack.2.weight 
Size: torch.Size([512, 512]) 
Values : tensor([[ 0.0321,  0.0153, -0.0320,  ..., -0.0059,  0.0207,  0.0300],
        [ 0.0250, -0.0329,  0.0413,  ..., -0.0231, -0.0255,  0.0002]],
       device='cuda:0', grad_fn=<SliceB